## Capstone 2 - Abalone Age Prediction
### Modeling
**Context**:

The age of abalone is determined by cutting the shell through the cone, staining it, and counting the number of rings through a microscope -- a boring and time-consuming task. Other measurements, which are easier to obtain, are used to predict the age. Further information, such as weather patterns and location (hence food availability) may be required to solve the problem.

_Credit: https://www.kaggle.com/rodolfomendes/abalone-dataset_

**Goal**: The goal of this capstone project is to build a regression model that can predict the age of an abalone shell by accurately predicting its ring count.


**Pre-processing & Training Data Development Objective**: Build two to three different models and identify the best one to predict the age of an abalone. 

In [157]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

from math import sqrt
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve, cross_val_score

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.dummy import DummyRegressor
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression

In [158]:
#Import abalone dataset
abalone_data = pd.read_csv('')

In [159]:
#One hot encode the 'Sex' column since it is categorical
one_hot = pd.get_dummies(abalone_data['Sex'])

# Drop 'Sex' column as it is now encoded
abalone_data = abalone_data.drop('Sex',axis = 1)

# Join the encoded df
abalone_data = abalone_data.join(one_hot)

#Confirm Sex is one hot encoded
abalone_data.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Age,F,I,M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,16.5,0,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,8.5,0,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,10.5,1,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,11.5,0,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,8.5,0,1,0


In [160]:
#Drop response variable
X = abalone_data.drop('Age', axis = 1)
y = abalone_data['Age']

In [180]:
#Split data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

## Linear Regression Model

_In statistics, linear regression is a linear approach for modelling the relationship between a scalar response and one or more explanatory variables (also known as dependent and independent variables). The case of one explanatory variable is called simple linear regression; for more than one, the process is called multiple linear regression. Wikipedia_

In [181]:
#Transform data so that it has a mean of 0 and std of 1
standardScale = StandardScaler()
standardScale.fit_transform(X)

#Split data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

#Create the linear regression model and fit it to training data
model_lin = linear_model.LinearRegression()
model_lin.fit(X_train, y_train)

LinearRegression()

In [163]:
#Train the model
y_pred_train_lin = model_lin.predict(X_train)

#Evaluate the performance
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_train, y_pred_train_lin)) #Mean squared error (MSE)
print('Root mean squared error (RMSE): %.2f' #Root mean squared error (RMSE)
      % sqrt(mean_squared_error(y_train, y_pred_train_lin)))
print('Coefficient of determination (R^2): %.2f' #Coefficient of determination
      % r2_score(y_train, y_pred_train_lin))

Mean squared error (MSE): 4.71
Root mean squared error (RMSE): 2.17
Coefficient of determination (R^2): 0.55


In [164]:
#Apply model to test set
y_pred_test_lin = model_lin.predict(X_test)

#Evaluate the performance
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_test, y_pred_test_lin)) #mean squared error (MSE)
print('Root mean squared error (RMSE): %.2f' #Root mean squared error (RMSE)
      % sqrt(mean_squared_error(y_test, y_pred_test_lin)))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(y_test, y_pred_test_lin)) 

Mean squared error (MSE): 5.11
Root mean squared error (RMSE): 2.26
Coefficient of determination (R^2): 0.49


## Ridge Regression Model

_Ridge regression is a method of estimating the coefficients of multiple-regression models in scenarios where linearly independent variables are highly correlated. Wikipedia_

In [165]:
#Create the ridge regression model and fit it to training data
model_ridge = Ridge()
model_ridge.fit(X_train, y_train)

Ridge()

In [166]:
#Train the model
y_pred_train_ridge = model_ridge.predict(X_train)

#Evaluate the performance
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_train, y_pred_train_ridge)) #Mean squared error (MSE)
print('Root mean squared error (RMSE): %.2f' #Root mean squared error (RMSE)
      % sqrt(mean_squared_error(y_train, y_pred_train_ridge)))
print('Coefficient of determination (R^2): %.2f' #Coefficient of determination
      % r2_score(y_train, y_pred_train_ridge))

Mean squared error (MSE): 4.75
Root mean squared error (RMSE): 2.18
Coefficient of determination (R^2): 0.55


In [167]:
#Apply model to test set
y_pred_test_ridge = model_ridge.predict(X_test)

#Evaluate the performance
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_test, y_pred_test_ridge)) #mean squared error (MSE)
print('Root mean squared error (RMSE): %.2f' #Root mean squared error (RMSE)
      % sqrt(mean_squared_error(y_test, y_pred_test_ridge)))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(y_test, y_pred_test_ridge)) 

Mean squared error (MSE): 5.07
Root mean squared error (RMSE): 2.25
Coefficient of determination (R^2): 0.49


## Random Forest Model

_Random forests or random decision forests are an ensemble learning method for classification, regression and other tasks that operates by constructing a multitude of decision trees at training time. For regression tasks, the mean or average prediction of the individual trees is returned. Wikipedia_

In [168]:
#Create the random forest model and fit it to training data
model_rf = RandomForestRegressor()
model_rf.fit(X_train, y_train)

RandomForestRegressor()

In [169]:
#Train the model
y_pred_train_rf = model_rf.predict(X_train)

#Evaluate the performance
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_train, y_pred_train_rf)) #Mean squared error (MSE)
print('Root mean squared error (RMSE): %.2f' #Root mean squared error (RMSE)
      % sqrt(mean_squared_error(y_train, y_pred_train_rf)))
print('Coefficient of determination (R^2): %.2f' #Coefficient of determination
      % r2_score(y_train, y_pred_train_rf))

Mean squared error (MSE): 0.65
Root mean squared error (RMSE): 0.81
Coefficient of determination (R^2): 0.94


In [170]:
#Apply model to test set
y_pred_test_rf = model_rf.predict(X_test)

#Evaluate the performance
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_test, y_pred_test_rf)) #mean squared error (MSE)
print('Root mean squared error (RMSE): %.2f' #Root mean squared error (RMSE)
      % sqrt(mean_squared_error(y_test, y_pred_test_rf)))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(y_test, y_pred_test_rf)) 

Mean squared error (MSE): 5.06
Root mean squared error (RMSE): 2.25
Coefficient of determination (R^2): 0.49


## Support Vector Machine (SVM) / Support Vector Regression (SVR)

_Given a set of training examples, each marked as belonging to one of two categories, an SVM training algorithm builds a model that assigns new examples to one category or the other, making it a non-probabilistic binary linear classifier. Wikipedia_ 

_Support Vector Regression is a supervised learning algorithm that is used to predict discrete values. Support Vector Regression uses the same principle as the SVMs. The basic idea behind SVR is to find the best fit line. In SVR, the best fit line is the hyperplane that has the maximum number of points. Towards Data Science_

In [171]:
#Create the SVR model and fit it to training data
model_svr = SVR()
model_svr.fit(X_train, y_train)

SVR()

In [172]:
#Train the model
y_pred_train_svr = model_svr.predict(X_train)

#Evaluate the performance
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_train, y_pred_train_svr)) #Mean squared error (MSE)
print('Root mean squared error (RMSE): %.2f' #Root mean squared error (RMSE)
      % sqrt(mean_squared_error(y_train, y_pred_train_svr)))
print('Coefficient of determination (R^2): %.2f' #Coefficient of determination
      % r2_score(y_train, y_pred_train_svr))

Mean squared error (MSE): 5.11
Root mean squared error (RMSE): 2.26
Coefficient of determination (R^2): 0.51


In [173]:
#Apply model to test set
y_pred_test_svr = model_svr.predict(X_test)

#Evaluate the performance
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_test, y_pred_test_svr)) #mean squared error (MSE)
print('Root mean squared error (RMSE): %.2f' #Root mean squared error (RMSE)
      % sqrt(mean_squared_error(y_test, y_pred_test_svr)))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(y_test, y_pred_test_svr)) 

Mean squared error (MSE): 5.31
Root mean squared error (RMSE): 2.31
Coefficient of determination (R^2): 0.47


## Gradient Boost Model

_Gradient boosting is a machine learning technique used in regression and classification tasks, among others. It gives a prediction model in the form of an ensemble of weak prediction models, which are typically decision trees.[1][2] When a decision tree is the weak learner, the resulting algorithm is called gradient-boosted trees; it usually outperforms random forest. Wikipedia_

In [174]:
#Create the gradient boost model and fit it to training data
model_gb = GradientBoostingRegressor()
model_gb.fit(X_train, y_train)

GradientBoostingRegressor()

In [175]:
#Train the model
y_pred_train_gb = model_gb.predict(X_train)

#Evaluate the performance
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_train, y_pred_train_gb)) #Mean squared error (MSE)
print('Root mean squared error (RMSE): %.2f' #Root mean squared error (RMSE)
      % sqrt(mean_squared_error(y_train, y_pred_train_gb)))
print('Coefficient of determination (R^2): %.2f' #Coefficient of determination
      % r2_score(y_train, y_pred_train_gb))

Mean squared error (MSE): 3.40
Root mean squared error (RMSE): 1.84
Coefficient of determination (R^2): 0.68


In [176]:
#Apply model to test set
y_pred_test_gb = model_gb.predict(X_test)

#Evaluate the performance
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_test, y_pred_test_gb)) #mean squared error (MSE)
print('Root mean squared error (RMSE): %.2f' #Root mean squared error (RMSE)
      % sqrt(mean_squared_error(y_test, y_pred_test_gb)))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(y_test, y_pred_test_gb)) 

Mean squared error (MSE): 4.92
Root mean squared error (RMSE): 2.22
Coefficient of determination (R^2): 0.51


## K Nearest Neighbors (KNN)

_KNN model is popularly used for non-linear regression in Machine Learning. KNN (K Nearest Neighbours) follows an easy implementation approach for non-linear regression in Machine Learning. KNN assumes that the new data point is similar to the existing data points. The new data point is compared to the existing categories and is placed under a relatable category. The average value of the k nearest neighbors is taken as the input in this algorithm. The neighbors in KNN models are given a particular weight that defines their contribution to the average value. Jigsaw Academy_

In [177]:
#Create the KNN model and fit it to training data
model_knn = KNeighborsRegressor(n_neighbors = 4)
model_knn.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=4)

In [178]:
#Train the model
y_pred_train_knn = model_knn.predict(X_train)

#Evaluate the performance
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_train, y_pred_train_knn)) #Mean squared error (MSE)
print('Root mean squared error (RMSE): %.2f' #Root mean squared error (RMSE)
      % sqrt(mean_squared_error(y_train, y_pred_train_knn)))
print('Coefficient of determination (R^2): %.2f' #Coefficient of determination
      % r2_score(y_train, y_pred_train_knn))

Mean squared error (MSE): 3.00
Root mean squared error (RMSE): 1.73
Coefficient of determination (R^2): 0.71


In [179]:
#Apply model to test set
y_pred_test_knn = model_knn.predict(X_test)

#Evaluate the performance
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_test, y_pred_test_knn)) #mean squared error (MSE)
print('Root mean squared error (RMSE): %.2f' #Root mean squared error (RMSE)
      % sqrt(mean_squared_error(y_test, y_pred_test_knn)))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(y_test, y_pred_test_knn)) 

Mean squared error (MSE): 5.40
Root mean squared error (RMSE): 2.32
Coefficient of determination (R^2): 0.46


## Model Evaluation Function

_Test multiple regression models on data and evaulate RMSE_

In [182]:
#Function to test all models

np.random.seed(10)
def rmse_cv(model, X_train, y):
    rmse =- (cross_val_score(model, X_train, y, scoring='neg_root_mean_squared_error', cv=5))
    return(rmse)

models = [LinearRegression(),
             Ridge(),
             SVR(),
             RandomForestRegressor(),
             GradientBoostingRegressor(),
             KNeighborsRegressor(n_neighbors = 4)]

names = ['LR','Ridge','SVR','RF','GB','KNN']

for model,name in zip(models,names):
    score = rmse_cv(model, X_train, y_train)
    print("{}    : {:.6f}, {:4f}".format(name, score.mean(), score.std()))

LR    : 2.270472, 0.044227
Ridge    : 2.245205, 0.066963
SVR    : 2.336136, 0.145117
RF    : 2.200266, 0.099129
GB    : 2.190296, 0.105288
KNN    : 2.333581, 0.113218
